In [2]:
import json
import pennylane as qml
import pennylane.numpy as np
# import jax
# from jax import numpy as jnp

In [84]:
import json
import pennylane as qml
import pennylane.numpy as np

WIRES = 2
LAYERS = 5
NUM_PARAMETERS = LAYERS * WIRES * 3
# ins = np.array([0.32158897156285354,-0.20689268438270836,0.12366748295758379,-0.11737425017261123,-0.20689268438270836,0.7747346055276305,-0.05159966365446514,0.08215539696259792,0.12366748295758379,-0.05159966365446514,0.5769050487087416,0.3853362904758938,-0.11737425017261123,0.08215539696259792,0.3853362904758938,0.3986256655167206])

# hamiltonian1 = np.array(ins,float).reshape((2 ** WIRES), (2 ** WIRES))
# params1 = np.random.random(NUM_PARAMETERS)

def variational_circuit(params,hamiltonian):
    
    parameters = params.reshape((LAYERS, WIRES, 3))
    qml.templates.StronglyEntanglingLayers(parameters, wires=range(WIRES))
    return qml.expval(qml.Hermitian(hamiltonian, wires = [0,1]))

def optimize_circuit(params,hamiltonian):

    angle = [params]
    dev = qml.device("default.qubit", wires = WIRES)# Initialize the device.

    circuit = qml.QNode(variational_circuit, dev, interface="autograd", diff_method="parameter-shift")

    cost = [circuit(params,hamiltonian)]
    # print(cost)
    opt = qml.AdagradOptimizer(stepsize=0.02, eps=1e-08) # Our optimizer!
   
    conv_tol = 1e-06 # Convergence threshold to stop our optimization procedure
    
    while(True):
        
        params,prev_cost = opt.step_and_cost(circuit,params,hamiltonian)
        params = params[0]
    #     print(params)
    #     print(prev_cost)
        cost.append(circuit(params,hamiltonian))
        angle.append(params)
    #     print("==================")
    # #     print(cost)
    #     print("**********************")
    #     print(prev_cost)
        conv = np.abs(cost[-1] - prev_cost)
        
        if conv <= conv_tol:
            break
       
        
    return np.array(cost[-1])

# q = optimize_circuit(params1,hamiltonian1)
# print(q)

# print("\n" f"Final value of the cost function = {cost[-1]:.8f} ")
# print("\n" f"Optimal value of the first circuit parameter =    "  + str(angle[-1][0]))
# print("\n" f"Optimal value of the second circuit parameter =    "  + str(angle[-1][1]))
    
      
    
    
    
    

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
       
    ins = np.array(json.loads(test_case_input), requires_grad = False)
    print(ins)
    hamiltonian = np.array(ins,float).reshape((2 ** WIRES), (2 ** WIRES))
    np.random.seed(1967)
    initial_params = np.random.random(NUM_PARAMETERS)
    out = str(optimize_circuit(initial_params,hamiltonian))    
    
    return out


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.isclose(solution_output, expected_output, rtol=5e-2)


# These are the public test cases
test_cases = [
    ('[0.863327072347624,0.0167108057202516,0.07991447085492759,0.0854049026262154,0.0167108057202516,0.8237963773906136,-0.07695947154193797,0.03131548733285282,0.07991447085492759,-0.07695947154193795,0.8355417021014687,-0.11345916130631205,0.08540490262621539,0.03131548733285283,-0.11345916130631205,0.758156886827099]', '0.61745341'),
    ('[0.32158897156285354,-0.20689268438270836,0.12366748295758379,-0.11737425017261123,-0.20689268438270836,0.7747346055276305,-0.05159966365446514,0.08215539696259792,0.12366748295758379,-0.05159966365446514,0.5769050487087416,0.3853362904758938,-0.11737425017261123,0.08215539696259792,0.3853362904758938,0.3986256655167206]', '0.00246488')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.863327072347624,0.0167108057202516,0.07991447085492759,0.0854049026262154,0.0167108057202516,0.8237963773906136,-0.07695947154193797,0.03131548733285282,0.07991447085492759,-0.07695947154193795,0.8355417021014687,-0.11345916130631205,0.08540490262621539,0.03131548733285283,-0.11345916130631205,0.758156886827099]'...
[ 0.86332707  0.01671081  0.07991447  0.0854049   0.01671081  0.82379638
 -0.07695947  0.03131549  0.07991447 -0.07695947  0.8355417  -0.11345916
  0.0854049   0.03131549 -0.11345916  0.75815689]
Correct!
Running test case 1 with input '[0.32158897156285354,-0.20689268438270836,0.12366748295758379,-0.11737425017261123,-0.20689268438270836,0.7747346055276305,-0.05159966365446514,0.08215539696259792,0.12366748295758379,-0.05159966365446514,0.5769050487087416,0.3853362904758938,-0.11737425017261123,0.08215539696259792,0.3853362904758938,0.3986256655167206]'...
[ 0.32158897 -0.20689268  0.12366748 -0.11737425 -0.20689268  0.77473461
 -0.051599

True

In [101]:
import json
import numpy as np

def initialize_state():
    """
    Prepare a qubit in state |0>.

    Returns:
        array[float]: the vector representation of state |0>.
    """


    # PREPARE THE STATE |0>
    return np.array([1,0])

def apply_u(U, state):
    """
    Apply the quantum operation U on the state

    Args:
        U (np.array(array(complex))): A (2,2) numpy array with complex entries 
        representing a unitary matrix.
        state (np.array(complex)): A (2,) numpy array with complex entries 
        representing a quantum state.
    
    Returns:
        (np.array(complex)): The state vector after applying U to state.
    """


    # Put your code here #
    return np.dot(U, state)

def measure_state(state, num_meas):
    """
    Measure a quantum state num_meas times.

    Args:
        state (np.array(complex)): A (2,) numpy array with complex entries
        representing a quantum state.
        num_meas(float): The number of computational basis measurements on state.
        
    Returns:
        (np.array(int)) A (num_meas,) numpy array of zeros and ones, representing
        measurement outcomes on the state
    """


    # Your code here #
        # Your code here #
    state_0 = initialize_state()
    state_1 = np.array([0,1])
    # meas_0 = np.outer(state_0, np.conj(state_0)) 
    # meas_1 = np.outer(state_1, np.conj(state_1))
#     temp_0 = apply_u( np.outer(state_0, np.conj(state_0)) , state )
#     temp_1 = apply_u( np.outer(state_1, np.conj(state_1)),  state )
    prob_0 = np.dot(np.conj(state), apply_u( np.outer(state_0, np.conj(state_0)) , state ))
    prob_1 = np.dot(np.conj(state), apply_u( np.outer(state_1, np.conj(state_1)),  state ) )
    return np.random.choice([0, 1], size=num_meas, p=[prob_0, prob_1])
    
def quantum_algorithm(U):
    """
    Use the functions above to implement the quantum algorithm described above.

    Try and do so using three lines of code or less!

    Args:
        U (np.array(array(complex))): A (2,2) numpy array with complex entries
        representing a unitary matrix.

    Returns:
        np.array(int): the measurement results after running the algorithm 20 times
    """


    # PREPARE THE STATE, APPLY U, THEN TAKE 20 MEASUREMENT SAMPLES
    ini_state = initialize_state()
    state = apply_u(U, ini_state)
    return measure_state(state, 20)

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    np.random.seed(0)
    ins = json.loads(test_case_input)
    output = quantum_algorithm(ins).tolist()
    return str(output)


def check(solution_output: str, expected_output: str) -> None:
    
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    
    initial_state = initialize_state()

    assert isinstance(
        initial_state, np.ndarray
    ), "The output of your initialize_state function should be a numpy array"

    assert np.allclose(
        initial_state, np.array([1, 0])
    ), "The output of your initialize_state function isn't quite right"

    U_test = [[0.70710678, 0.70710678], [0.70710678, -0.70710678]]

    assert np.allclose(
        apply_u(U_test, np.array([1, 0])), [0.70710678, 0.70710678]
    ), "Your output of apply_u isn't quite right"

    sample_list = measure_state([0.70710678, 0.70710678], 100).tolist()

    assert (
        sample_list.count(0) + sample_list.count(1) == 100
    ), "Your measure_state function isn't quite right"

    sample_list_2 = quantum_algorithm(U_test).tolist()

    assert (
        sample_list_2.count(0) + sample_list_2.count(1) == 20
    ), "Your quantum_algorithm function isn't quite right"
    
    assert np.allclose(solution_output, expected_output)


# These are the public test cases
test_cases = [
    ('[[0.70710678,  0.70710678], [0.70710678, -0.70710678]]', '[1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]'),
    ('[[0.8660254, -0.5],[0.5, 0.8660254]]', '[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[0.70710678,  0.70710678], [0.70710678, -0.70710678]]'...
Correct!
Running test case 1 with input '[[0.8660254, -0.5],[0.5, 0.8660254]]'...
Correct!


In [99]:
U_test = [[0.70710678, 0.70710678], [0.70710678, -0.70710678]]
row_vector = np.array([1,0])
np.outer(row_vector, np.conj(row_vector))

array([[1, 0],
       [0, 0]])

array([1, 0])